## SRJ

In [ ]:
import pennylane as qml
from pennylane import qchem
from pennylane import numpy as np
from itertools import chain
import time
import re
from scipy.optimize import minimize
ash_excitation = []
energies = []
excitations= []

X = qml.PauliX
Y = qml.PauliY
Z = qml.PauliZ
I = qml.Identity



print('Using BFGS + charge and mult')
print('I changed the wires1 and wires2')


def ags_exact(symbols, coordinates, active_electrons, active_orbitals, adapt_it, shots = None):
    print('Using active space, check if you change the H accordingly')
    core, active = qml.qchem.active_space(electrons, orbitals, active_electrons=active_electrons, active_orbitals=active_orbitals) # Spatial orbitals
    print('core orbitals:', core)
    print('active orbitals:', active)
    H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates, basis="sto-3g", method = 'pyscf',charge = 0, mult =1,active_electrons=active_electrons, active_orbitals=active_orbitals)
    print(H)
    hf_state = qchem.hf_state(active_electrons, qubits)
    print('HF state:', hf_state)
    #Calculation of HF state
    dev = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev)
    def circuit(hf_state, active_electrons, qubits, H):
        #print('Updated hf_state is', hf_state)  
        qml.BasisState(hf_state, wires=range(qubits))
        return qml.expval(H)   #Calculating the expectation value of the Hamiltonian
    
    # Commutator calculation for HF state
    @qml.qnode(dev)
    def commutator_0(H,w, k):  #H is the Hamiltonian, w is the operator, k is the basis state - HF state
        qml.BasisState(k, wires=range(qubits))
        res = qml.commutator(H, w)   #Calculating the commutator
        return qml.expval(res)
    
    # Commutator calculation for other states except HF state
    @qml.qnode(dev)
    def commutator_1(H,w, k): #H is the Hamiltonian, w is the operator, k is the basis state
        qml.StatePrep(k, wires=range(qubits))
        res = qml.commutator(H, w) #Calculating the commutator
        return qml.expval(res)

    #Energy calculation 
    @qml.qnode(dev)
    def ash(params, ash_excitation, hf_state, H):
        [qml.PauliX(i) for i in np.nonzero(hf_state)[0]]  #Appln of HF state
        for i, excitation in enumerate(ash_excitation):
            if len(ash_excitation[i]) == 4:
                qml.FermionicDoubleExcitation(weight=params[i], wires1=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)), wires2=list(range(ash_excitation[i][2], ash_excitation[i][3] + 1)))
            elif len(ash_excitation[i]) == 2:
                qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))
        return qml.expval(H)  #Calculating the expectation value of the Hamiltonian
    
    # Calculation of New state, same as the above function but with the state return
    dev1 = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev1)
    def new_state(hf_state, ash_excitation, params):
        [qml.PauliX(i) for i in np.nonzero(hf_state)[0]] #Applying the HF state
        for i, excitations in enumerate(ash_excitation):
            if len(ash_excitation[i]) == 4:
                qml.FermionicDoubleExcitation(weight=params[i], wires1=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)), wires2=list(range(ash_excitation[i][2], ash_excitation[i][3] + 1)))
            elif len(ash_excitation[i]) == 2:
                qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))
        return qml.state()
    

    
    
    def cost(params):
        energy = ash(params, ash_excitation, hf_state, H)
        return energy

    def callback(params):
        print(f"Current parameters: {params}")
        print(f"Current cost: {cost(params)}\n")
    

    print('HF state is', circuit(hf_state, active_electrons, qubits, H))
    singles, doubles = qml.qchem.excitations(active_electrons, qubits)
    print('Doubles:',doubles)
    op1 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "-"}) for x in singles]
    op2 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "+", (2, x[2]): "-", (3, x[3]): "-"})for x in doubles]
    print('Op2:', op2)
    operator_pool = (op1) + (op2)  #Operator pool - Singles and Doubles
    print('Total excitations are', len(operator_pool))
    states = [hf_state]
    params = np.zeros(len(ash_excitation), requires_grad=True) 

    null_state = np.zeros(qubits,int)
    print('Null state is', null_state)

    for j in range(adapt_it):
        print('The adapt iteration now is', j)  #Adapt iteration
        max_value = float('-inf')
        max_operator = None
        k = states[-1] if states else hf_state  # if states is empty, fall back to hf_state
       
        for i in operator_pool:
            #print('The current excitation operator is', i)   #Current excitation operator - fermionic one
            w = qml.fermi.jordan_wigner(i)  #JW transformation
            if np.array_equal(k, hf_state): # If the current state is the HF state
                current_value = abs(2*(commutator_0(H, w, k)))      #Commutator calculation is activated  
            else:
                current_value = abs(2*(commutator_1(H, w, k)))      #For other states, commutator calculation is activated
            print(f'The expectation value of {i} is', current_value)
            if current_value > max_value:
                max_value = current_value
                max_operator = i

        #print(f"The highest operator value is {max_value} for operator {max_operator}")  #Highest operator value


        indices_str = re.findall(r'\d+', str(max_operator))
        excitations = [int(index) for index in indices_str]
        print('Highest gradient excitation is', excitations)
        ash_excitation.append(excitations) #Appending the excitations to the ash_excitation

        params = np.append(params, 0.0)  #Parameters initialization



        #Energy calculation
        result = minimize(cost, params, method='BFGS', callback=callback, tol = 1e-12, options = {'disp': False, 'maxiter': 1e8})

        print("Final updated parameters:", result.x)
        print("Final cost:", result.fun)

        params= (result.x)
        energies.append(result.fun)


        ostate = new_state(hf_state, ash_excitation, params)
        #print(qml.draw(new_state, max_length=100)(hf_state,ash_excitation,params))
        gs_state = ostate
        states.append(ostate)
        
    return gs_state, params, ash_excitation, qubits, H

## So if you want the state, return the ostate and not states




symbols  = [ 'N', 'N']
print('N2-1.10-GS-COBYLA-Added charge&mult-(6,6)')
r_bohr = 1.8897259886 
coordinates = np.array([[0.0,0.0, 0.0], [0.0, 0.0, 1.10*r_bohr]])



electrons = 14  # 7 from N and 7 from N
orbitals = 10
charge = 0

active_electrons = 6
active_orbitals = 6


gs_state, params, ash_excitation, qubits, H = ags_exact(symbols, coordinates, active_electrons, active_orbitals, shots = None, adapt_it=1) #1 is used for params



print('The params after GS is',params)
print('Ash excitation after gs state:', ash_excitation)





Using BFGS + charge and mult
I changed the wires1 and wires2
N2-1.10-GS-COBYLA-Added charge&mult-(6,6)
Using active space, check if you change the H accordingly
core orbitals: [0, 1, 2, 3]
active orbitals: [4, 5, 6, 7, 8, 9]
-104.77699665549983 * I([0, 2, 4, 6, 7, 8, 10, 1, 3, 5, 9, 11]) + 0.13491335839891902 * Z(0) + 0.1349133583989192 * Z(2) + 0.1290154134311182 * (Z(0) @ Z(2)) + 0.12384971850799134 * Z(4) + 0.12242614830946354 * (Z(0) @ Z(4)) + -0.08793748500011 * Z(6) + 0.09986327064850209 * (Z(0) @ Z(6)) + -0.0005233422646846878 * (Z(0) @ Y(6) @ Z(7) @ Y(8)) + -0.0005233422646846878 * (Z(0) @ X(6) @ Z(7) @ X(8)) + -0.08793748500011021 * Z(8) + 0.1305770919985938 * (Z(0) @ Z(8)) + -0.4701696087061984 * Z(10) + 0.14615606164779144 * (Z(0) @ Z(10)) + 0.1349133583989189 * Z(1) + 0.14700833039958525 * (Z(0) @ Z(1)) + 0.0059976389894889825 * (Y(0) @ X(1) @ X(2) @ Y(3)) + -0.0059976389894889825 * (Y(0) @ Y(1) @ X(2) @ X(3)) + -0.0059976389894889825 * (X(0) @ X(1) @ Y(2) @ Y(3)) + 0.00599

/Users/s.poyyapakkam/miniconda3/envs/sri/lib/python3.12/site-packages/pennylane_lightning/core/_serialize.py:353: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)


The expectation value of a⁺(1) a(11) is 0.0
The expectation value of a⁺(2) a(6) is 0.0
The expectation value of a⁺(2) a(8) is 0.0
The expectation value of a⁺(2) a(10) is 0.0
The expectation value of a⁺(3) a(7) is 0.0
The expectation value of a⁺(3) a(9) is 0.0
The expectation value of a⁺(3) a(11) is 0.0
The expectation value of a⁺(4) a(6) is 0.0
The expectation value of a⁺(4) a(8) is 0.0
The expectation value of a⁺(4) a(10) is 0.0
The expectation value of a⁺(5) a(7) is 0.0
The expectation value of a⁺(5) a(9) is 0.0
The expectation value of a⁺(5) a(11) is 0.0
The expectation value of a⁺(0) a⁺(1) a(6) a(7) is 0.3752256976595048
The expectation value of a⁺(0) a⁺(1) a(6) a(9) is 0.005745108687194703
The expectation value of a⁺(0) a⁺(1) a(6) a(11) is 0.0
The expectation value of a⁺(0) a⁺(1) a(7) a(8) is 0.005745108687194703
The expectation value of a⁺(0) a⁺(1) a(7) a(10) is 0.0
The expectation value of a⁺(0) a⁺(1) a(8) a(9) is 0.03805774130942971
The expectation value of a⁺(0) a⁺(1) a(8) a(1

## Gradient calculation

In [2]:

dev = qml.device("lightning.qubit", wires=qubits)
@qml.qnode(dev)
def g_0(H,w, hf_state):  #H is the Hamiltonian, w is the operator, k is the basis state - HF state
    qml.BasisState(hf_state, wires=range(qubits))
    res = qml.commutator(H, w)   #Calculating the commutator
    return qml.expval(res)

qubits=12
hf_state = np.array([1, 1, 1, 1 ,1, 1, 0, 0, 0, 0, 0, 0])
singles, doubles = qml.qchem.excitations(active_electrons, qubits)
print('Doubles:',doubles)
op1 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "-"}) for x in singles]
op2 =  [qml.fermi.FermiWord({(0, x[0]): "-", (1, x[1]): "-", (2, x[2]): "+", (3, x[3]): "+"})for x in doubles]
print('Op2:', op2)

operator_pool = (op1) + (op2)  #Operator pool - Singles and Doubles
print('Total excitations are', len(operator_pool))


max_value = float('-inf')
max_operator = None


       
for i in operator_pool:
    print('The current excitation operator is', i)   #Current excitation operator - fermionic one
    w = qml.fermi.jordan_wigner(i)  #JW transformation
    current_value = abs(2*(g_0(H, w, hf_state)))
    print(f'The expectation value of {i} is', current_value)
    if current_value > max_value:
        max_value = current_value
        max_operator = i

print(f"The highest operator value is {max_value} for operator {max_operator}")  #Highest operator value

Doubles: [[0, 1, 6, 7], [0, 1, 6, 9], [0, 1, 6, 11], [0, 1, 7, 8], [0, 1, 7, 10], [0, 1, 8, 9], [0, 1, 8, 11], [0, 1, 9, 10], [0, 1, 10, 11], [0, 2, 6, 8], [0, 2, 6, 10], [0, 2, 8, 10], [0, 3, 6, 7], [0, 3, 6, 9], [0, 3, 6, 11], [0, 3, 7, 8], [0, 3, 7, 10], [0, 3, 8, 9], [0, 3, 8, 11], [0, 3, 9, 10], [0, 3, 10, 11], [0, 4, 6, 8], [0, 4, 6, 10], [0, 4, 8, 10], [0, 5, 6, 7], [0, 5, 6, 9], [0, 5, 6, 11], [0, 5, 7, 8], [0, 5, 7, 10], [0, 5, 8, 9], [0, 5, 8, 11], [0, 5, 9, 10], [0, 5, 10, 11], [1, 2, 6, 7], [1, 2, 6, 9], [1, 2, 6, 11], [1, 2, 7, 8], [1, 2, 7, 10], [1, 2, 8, 9], [1, 2, 8, 11], [1, 2, 9, 10], [1, 2, 10, 11], [1, 3, 7, 9], [1, 3, 7, 11], [1, 3, 9, 11], [1, 4, 6, 7], [1, 4, 6, 9], [1, 4, 6, 11], [1, 4, 7, 8], [1, 4, 7, 10], [1, 4, 8, 9], [1, 4, 8, 11], [1, 4, 9, 10], [1, 4, 10, 11], [1, 5, 7, 9], [1, 5, 7, 11], [1, 5, 9, 11], [2, 3, 6, 7], [2, 3, 6, 9], [2, 3, 6, 11], [2, 3, 7, 8], [2, 3, 7, 10], [2, 3, 8, 9], [2, 3, 8, 11], [2, 3, 9, 10], [2, 3, 10, 11], [2, 4, 6, 8], [2, 4, 6

In [ ]:
import pennylane as qml
import numpy as np
hf_state = np.array([1, 1, 0,0])
qubits = 4

In [ ]:
dev = qml.device("lightning.qubit", wires=qubits)
@qml.qnode(dev)
def summa():
    qml.BasisState(hf_state, wires=range(qubits))
    qml.DoubleExcitation(np.pi, wires= [0,1,2,3])
    return qml.state()
summa()

In [ ]:
hf_state = np.array([1, 1, 1, 1 ,1, 1, 0, 0, 0, 0, 0, 0])
qubits=12
excitation = np.array([0,3,7,8])
print(len(excitation))
params = qml.numpy.array(0.0, requires_grad=True)
print(params)

dev = qml.device("lightning.qubit", wires=qubits)
@qml.qnode(dev)

def cost_fn(params):
    [qml.PauliX(i) for i in np.nonzero(hf_state)[0]]
    print('wires2:', list(range(excitation[0], excitation[1] + 1)))
    print('wires1:', list(range(excitation[2], excitation[3] + 1)))
    qml.FermionicDoubleExcitation(weight=params, wires1=list(range(excitation[0], excitation[1] + 1)), wires2=list(range(excitation[2], excitation[3] + 1)))
    return qml.expval(H)

optimizer = qml.GradientDescentOptimizer(stepsize=0.5)
for i in range(500):
    params, energy = optimizer.step_and_cost(cost_fn, params)

print('Final energy:', energy)
print('Theta1:', params)

In [ ]:
#wires2 = -107.4971187218739